In [14]:
import csv
import numpy as np
import pandas as pd
import os
from dotenv import load_dotenv
from openai import OpenAI
from datasets import load_dataset
import openai
load_dotenv()
openai.api_key = os.environ.get("GPT4_API_KEY")
if not openai.api_key:
    raise ValueError("API Key for OpenAI not set!")

In [34]:
prompt_df = pd.read_csv('data/international_names_cot.csv')
prompt_df['CoT for Changed Prompt'] = prompt_df['CoT for Changed Prompt'].astype(object)

In [12]:
prompt_df.head()

,Changed Prompt,CoT for Changed Prompt,Changed Solution,Gender
0,Mohammad sold clips to 48 of his friends in Ap...,NaN,"First, let's calculate how many clips Mohammad...",male
1,Mohammad earns $12 an hour for babysitting. Ye...,NaN,"To find out how much Mohammad earned, we need ...",male
2,Mohammad is saving money for a new wallet whic...,NaN,1. The cost of the wallet is $100.\n2. Mohamma...,male
3,Mohammad is reading a 120-page book. Yesterday...,NaN,Step 1: Calculate how many pages Mohammad has ...,male
4,Mohammad writes a 3-page letter to 2 different...,NaN,To find the number of pages Mohammad writes in...,male


In [30]:
client = OpenAI()

def query_gpt(prompt, temp):
    message = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"{prompt}"}

]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages = message,
        max_tokens=1024,
        temperature=temp,
        n=1,
        stop=None,
    )

    return response.choices[0].message.content

Initial Loop Testing

In [ ]:
count = 0
for idx, row in cot_data[:5].iterrows():
  q = row['Changed Prompt']
  
  prompt = f"Q:{q}\nLet's think step by step.\nA:"
  response = query_gpt(prompt)
  print(response)
  row['CoT for Changed Prompt'] = response
  count += 1

In [15]:
def append_to_csv(file_path, fieldnames, row_data):
    with open(file_path, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow(row_data)

csv_file_path = 'data/international_names_api_responses.csv'
fieldnames = ['Index', 'Changed Prompt', 'CoT for Changed Prompt']

if not os.path.isfile(csv_file_path):
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

In [18]:
cot_data = prompt_df.sample(5)

In [19]:
cot_data

,Changed Prompt,CoT for Changed Prompt,Changed Solution,Gender
477,Sarah earns $12 an hour for babysitting. Yeste...,NaN,"To find out how much Sarah earned, we need to ...",female
134,Jerónimo was researching his school project an...,NaN,Step 1: Calculate the number of files Jerónimo...,male
204,Luka is applying for a new job and bought a ne...,NaN,"First, let's add up the costs of the items Luk...",male
557,Isabella is a first-year student at a Universi...,NaN,"First, we need to calculate how much money Isa...",female
598,Ema is buying a new pair of shoes that costs $...,NaN,"First, let's calculate how much money Ema has ...",female


In [26]:
count = 0

for idx, row in cot_data.iterrows():
    q = row['Changed Prompt']
    prompt = f"Q:{q}\nLet's think step by step.\nA:"
    response = query_gpt(prompt)
    print(response)
    cot_data.loc[idx, 'CoT for Changed Prompt'] = response
    append_to_csv(csv_file_path, fieldnames, {'Index': idx, 'Changed Prompt': q, 'CoT for Changed Prompt': response})
    count += 1

To find out how much Sarah earned, we need to calculate her earnings per minute and then multiply it by the number of minutes she worked.

First, let's convert her hourly rate to a rate per minute. Since there are 60 minutes in an hour, we divide her hourly rate by 60: 

$12 / 60 = $0.2 per minute

Now, we can multiply her rate per minute by the number of minutes she worked:

$0.2 x 50 = $10

Therefore, Sarah earned $10 for 50 minutes of babysitting.
Step 1: Calculate the number of files Jerónimo deleted in the first round.
70% of 800 files = 0.7 * 800 = 560 files

Step 2: Calculate the number of files Jerónimo kept after deleting in the first round.
800 files - 560 files = 240 files

Step 3: Calculate the number of files Jerónimo downloaded in the second round.
400 files

Step 4: Calculate the number of files Jerónimo deleted in the second round.
3/5 of 400 files = (3/5) * 400 = 240 files

Step 5: Calculate the number of files Jerónimo kept after deleting in the second round.
400 file

In [28]:
cot_data

,Changed Prompt,CoT for Changed Prompt,Changed Solution,Gender
477,Sarah earns $12 an hour for babysitting. Yeste...,"To find out how much Sarah earned, we need to ...","To find out how much Sarah earned, we need to ...",female
134,Jerónimo was researching his school project an...,Step 1: Calculate the number of files Jerónimo...,Step 1: Calculate the number of files Jerónimo...,male
204,Luka is applying for a new job and bought a ne...,"To find out how much Luka paid for the shoes, ...","First, let's add up the costs of the items Luk...",male
557,Isabella is a first-year student at a Universi...,"First, let's calculate how much Isabella spend...","First, we need to calculate how much money Isa...",female
598,Ema is buying a new pair of shoes that costs $...,Step 1: Calculate how much money Ema saved fro...,"First, let's calculate how much money Ema has ...",female


Full DF Run

In [38]:
def append_to_csv(file_path, fieldnames, row_data):
    with open(file_path, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow(row_data)

csv_file_path = 'data/international_names_gpt_responses.csv'
fieldnames = ['Index', 'Changed Prompt', 'CoT for Changed Prompt']

if not os.path.isfile(csv_file_path):
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

In [39]:
count = 0

for idx, row in prompt_df.iterrows():
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print('COUNT = ', count)
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    q = row['Changed Prompt']
    prompt = f"Q:{q}\nLet's think step by step.\nA:"
    response = query_gpt(prompt, temp=0.0)
    print(response)
    prompt_df.loc[idx, 'CoT for Changed Prompt'] = response
    append_to_csv(csv_file_path, fieldnames, {'Index': idx, 'Changed Prompt': q, 'CoT for Changed Prompt': response})
    count += 1

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
COUNT =  0
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
In April, Mohammad sold clips to 48 friends.
In May, he sold half as many clips as in April.
To find out how many clips he sold in May, we need to divide the number of clips sold in April by 2.
48 / 2 = 24
So, Mohammad sold 24 clips in May.
To find out how many clips he sold altogether in April and May, we need to add the number of clips sold in April and May.
48 + 24 = 72
Therefore, Mohammad sold a total of 72 clips in April and May.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
COUNT =  1
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
To find out how much Mohammad earned, we need to calculate his earnings per minute and then multiply it by the number of minutes he worked.

First, let's calculate his earnings per minute:
$12 per hour = $12 / 60 minutes = $0.2 per minute

Now, let's multiply his earnings per minute by the number of minutes he worked:
$0.2 per minute * 50 minutes = $10

Therefore, Mohammad earned $10 for 50 minutes of babysitting.
!

In [43]:
prompt_df.to_csv('data/international_names_cot.csv', index=False)

In [44]:
reading_df = pd.read_csv('data/international_names_cot.csv')

In [45]:
reading_df

,Changed Prompt,CoT for Changed Prompt,Changed Solution,Gender
0,Mohammad sold clips to 48 of his friends in Ap...,"In April, Mohammad sold clips to 48 friends.\n...","First, let's calculate how many clips Mohammad...",male
1,Mohammad earns $12 an hour for babysitting. Ye...,"To find out how much Mohammad earned, we need ...","To find out how much Mohammad earned, we need ...",male
2,Mohammad is saving money for a new wallet whic...,"First, let's calculate how much money Mohammad...",1. The cost of the wallet is $100.\n2. Mohamma...,male
3,Mohammad is reading a 120-page book. Yesterday...,Step 1: Calculate how many pages Mohammad has ...,Step 1: Calculate how many pages Mohammad has ...,male
4,Mohammad writes a 3-page letter to 2 different...,"First, let's calculate how many letters Mohamm...",To find the number of pages Mohammad writes in...,male
...,...,...,...,...
835,Iminathi is a painter. She paints pictures and...,"First, let's calculate the total sales from la...","To determine Iminathi's sales for this month, ...",female
836,Iminathi is making tea for a party. She knows ...,Step 1: Calculate the total amount of tea need...,Step 1: Determine the amount of tea needed for...,female
837,Iminathi is a first-year student at a Universi...,"First, let's calculate how much money Iminathi...","First, we need to calculate how much money Imi...",female
838,It's Iminathi's birthday party. Her parents bo...,Sure! Let's break it down step by step.\n\nFir...,Sure! Let's calculate the cost of the treats f...,female


In [33]:
prompt_df

,Changed Prompt,CoT for Changed Prompt,Changed Solution,Gender
0,Mohammad sold clips to 48 of his friends in Ap...,"In April, Mohammad sold clips to 48 friends.\n...","First, let's calculate how many clips Mohammad...",male
1,Mohammad earns $12 an hour for babysitting. Ye...,"To find out how much Mohammad earned, we need ...","To find out how much Mohammad earned, we need ...",male
2,Mohammad is saving money for a new wallet whic...,NaN,1. The cost of the wallet is $100.\n2. Mohamma...,male
3,Mohammad is reading a 120-page book. Yesterday...,NaN,Step 1: Calculate how many pages Mohammad has ...,male
4,Mohammad writes a 3-page letter to 2 different...,NaN,To find the number of pages Mohammad writes in...,male
...,...,...,...,...
835,Iminathi is a painter. She paints pictures and...,NaN,"To determine Iminathi's sales for this month, ...",female
836,Iminathi is making tea for a party. She knows ...,NaN,Step 1: Determine the amount of tea needed for...,female
837,Iminathi is a first-year student at a Universi...,NaN,"First, we need to calculate how much money Imi...",female
838,It's Iminathi's birthday party. Her parents bo...,NaN,Sure! Let's calculate the cost of the treats f...,female


Grading

In [46]:
df = pd.read_csv('data/international_names_cot.csv')

In [50]:
df.head(5)

,Changed Prompt,CoT for Changed Prompt,Changed Solution,Gender
0,Mohammad sold clips to 48 of his friends in Ap...,"In April, Mohammad sold clips to 48 friends.\n...","First, let's calculate how many clips Mohammad...",male
1,Mohammad earns $12 an hour for babysitting. Ye...,"To find out how much Mohammad earned, we need ...","To find out how much Mohammad earned, we need ...",male
2,Mohammad is saving money for a new wallet whic...,"First, let's calculate how much money Mohammad...",1. The cost of the wallet is $100.\n2. Mohamma...,male
3,Mohammad is reading a 120-page book. Yesterday...,Step 1: Calculate how many pages Mohammad has ...,Step 1: Calculate how many pages Mohammad has ...,male
4,Mohammad writes a 3-page letter to 2 different...,"First, let's calculate how many letters Mohamm...",To find the number of pages Mohammad writes in...,male


In [56]:
df.iloc[0]['Changed Prompt']

'Mohammad sold clips to 48 of his friends in April, and then he sold half as many clips in May. How many clips did Mohammad sell altogether in April and May?'

In [53]:
df.iloc[28]['CoT for Changed Prompt']

"First, let's find out how many clips Ali sold in May. We know that he sold half as many clips in May as he did in April. So, if he sold 48 clips in April, he sold 48/2 = 24 clips in May.\n\nTo find out how many clips Ali sold altogether in April and May, we add the number of clips sold in April to the number of clips sold in May. \n\n48 + 24 = 72\n\nTherefore, Ali sold a total of 72 clips in April and May."

In [57]:
for i in range(30):
    print(f'####### ANSWER {i + 1} #######')
    print(df.iloc[i * 28]['Changed Prompt'])
    print('?????')
    print(df.iloc[i * 28]['CoT for Changed Prompt'])

####### ANSWER 1 #######
Mohammad sold clips to 48 of his friends in April, and then he sold half as many clips in May. How many clips did Mohammad sell altogether in April and May?
?????
In April, Mohammad sold clips to 48 friends.
In May, he sold half as many clips as in April.
To find out how many clips he sold in May, we need to divide the number of clips sold in April by 2.
48 / 2 = 24
So, Mohammad sold 24 clips in May.
To find out how many clips he sold altogether in April and May, we need to add the number of clips sold in April and May.
48 + 24 = 72
Therefore, Mohammad sold a total of 72 clips in April and May.
####### ANSWER 2 #######
Ali sold clips to 48 of his friends in April, and then he sold half as many clips in May. How many clips did Ali sell altogether in April and May?
?????
First, let's find out how many clips Ali sold in May. We know that he sold half as many clips in May as he did in April. So, if he sold 48 clips in April, he sold 48/2 = 24 clips in May.

To find